In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
# import packages 

import rasterio as rio
from rasterio.transform import from_origin

In [2]:
files = sorted(getFilelist('/data/Aldhani/eoagritwin/et/Sentinel3/raw_not_masked', '.nc'))
year = 2024
# get a subset of files for that year
yearFiles = [file for file in files if int(file.split('/')[-1].split('_')[-1][0:4]) == year]

# make dictionary that stores .tif names and related accDates to easier search to close observations with S2 data
keys = ['filename', 'accDateTimes']
vals = [list() for i in range(len(keys))]
lookUp = dict(zip(keys, vals))


In [ ]:
# set storPath for exported tiffs
storPath = '/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/'
yearCont = []
# loop over files and export to .tif at location storPath
for file in yearFiles:

    accDateTimes = getAllDatesS3(file) # possible to take annual subset if entire files list would be passed here
    convertNCtoTIF(file, storPath + 'LST/daily_observations_all_not_masked/', file.split('/')[-1].split('.')[0] + '.tif', accDateTimes, False, True)

    # write filenames and corresponding dates into dictionary for easier datesearch later
    lookUp['filename'].append(file.split('.nc')[0])
    lookUp['accDateTimes'].append(accDateTimes)

    dat = getDataFromNC(file)
    monthCont = []
    df = pd.Series(accDateTimes)
    counts_per_day = df.dt.floor("D").value_counts().sort_index()
    # vectors for indexing over days
    t2 = np.cumsum(counts_per_day)
    t1 = np.insert(t2, 0 ,0)

    for l in range(len(counts_per_day)):
        monthCont.append(np.any(~np.isnan(dat[:, :, t1[l]:t2[l]]),axis=2)) # minimum dail obs
    exportNCarrayDerivatesInt(file, storPath + 'Analytics/', f'Minimum_DailyObservations_{('_').join(file.split('_')[-1].split('-')[:2])}.tif', 'monthly_sum_of_daily_obs', np.nansum(np.dstack((monthCont)), axis = 2), True)
    yearCont.append(np.nansum(np.dstack((monthCont)), axis = 2))
exportNCarrayDerivatesInt(file, storPath + 'Analytics/', f'Minimum_DailyObservations_{file.split('_')[-1].split('-')[0]}.tif', 'annual_sum_of_daily_obs', np.nansum(np.dstack((yearCont)), axis = 2), True)


/home/potzschf/mambaforge/envs/evapo_sentinelhub/lib/python3.13/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [3]:
# set storPath for exported tiffs
storPath = '/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/'
yearCont = []
# loop over files and export to .tif at location storPath
file = yearFiles[0]

accDateTimes = getAllDatesS3(file) # possible to take annual subset if entire files list would be passed here
#convertNCtoTIF(file, storPath + 'LST/daily_observations_all_not_masked/', file.split('/')[-1].split('.')[0] + '.tif', accDateTimes, False, True)

dat = getDataFromNC(file)
monthCont = []
df = pd.Series(accDateTimes)
counts_per_day = df.dt.floor("D").value_counts().sort_index()
# vectors for indexing over days
t2 = np.cumsum(counts_per_day)
t1 = np.insert(t2, 0 ,0)

for l in range(len(counts_per_day)):
    monthCont.append(np.any(~np.isnan(dat[:, :, t1[l]:t2[l]]),axis=2)) # minimum dail obs
#     exportNCarrayDerivatesInt(file, storPath + 'Analytics/', f'Minimum_DailyObservations_{('_').join(file.split('_')[-1].split('-')[:2])}.tif', 'monthly_sum_of_daily_obs', np.nansum(np.dstack((monthCont)), axis = 2), True)
#     yearCont.append(np.nansum(np.dstack((monthCont)), axis = 2))
# exportNCarrayDerivatesInt(file, storPath + 'Analytics/', f'Minimum_DailyObservations_{file.split('_')[-1].split('-')[0]}.tif', 'annual_sum_of_daily_obs', np.nansum(np.dstack((yearCont)), axis = 2), True)

/tmp/ipykernel_1692892/180451474.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  monthCont.append(np.any(~np.isnan(dat[:, :, t1[l]:t2[l]]),axis=2)) # minimum dail obs


In [10]:
np.unique(np.nansum(np.dstack((monthCont)), axis = 2))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [14]:
counts_per_day

2024-12-01    12
2024-12-02    10
2024-12-03    10
2024-12-04    10
2024-12-05    12
2024-12-06    10
2024-12-07    10
2024-12-08    11
2024-12-09    11
2024-12-10    10
2024-12-11     9
2024-12-12    11
2024-12-13    11
2024-12-14    10
2024-12-15    10
2024-12-16    11
2024-12-17    11
2024-12-18    10
2024-12-19    10
2024-12-20    11
2024-12-21    11
2024-12-22    10
2024-12-23    10
2024-12-24    11
2024-12-25    11
2024-12-26    10
2024-12-27    10
2024-12-28    12
2024-12-29    10
2024-12-30    10
2024-12-31    10
Name: count, dtype: int64

[[1 1 0 0 0]
 [1 1 0 0 0]
 [0 0 1 1 0]
 [0 0 1 1 0]
 [0 0 0 0 0]]
